# Tutorial 5 CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np

print(f"TensorFlow Version: {tf.__version__}")

In [ ]:
# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normalize the pixel values to the range [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Class names for CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Check the shapes of the data
print(f"Train images shape: {train_images.shape}")
print(f"Test images shape: {test_images.shape}")

In [ ]:
plt.figure(figsize=(10, 10)) # Set figure size for 5 images stacked vertically (10, 20) is from source [cite: 28]
for i in range(5):
    plt.subplot(5, 1, i + 1) # 5 rows, 1 column, i-th plot
    plt.xticks([]) # Remove x-axis ticks [cite: 31, 34]
    plt.yticks([]) # Remove y-axis ticks [cite: 32, 35]
    plt.grid(False) # Disable grid [cite: 33, 36]
    plt.imshow(train_images[i]) # Show the image [cite: 37]
    # Use the first element [0] since train_labels is a 2D array (N, 1)
    plt.xlabel(class_names[train_labels[i][0]]) # Label with class name [cite: 38]

plt.tight_layout()
plt.show()

In [ ]:
# Create the Sequential model
model = models.Sequential([
    # First Conv2D layer
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    # First MaxPooling2D layer
    layers.MaxPooling2D((2, 2)),
    # Second Conv2D layer
    layers.Conv2D(64, (3, 3), activation='relu'),
    # Second MaxPooling2D layer
    layers.MaxPooling2D((2, 2)),
    # Third Conv2D layer
    layers.Conv2D(64, (3, 3), activation='relu'),

    # Flatten the output for the Dense layers
    layers.Flatten(),
    # Fully connected layer
    layers.Dense(64, activation='relu'),
    # Output layer (10 neurons for 10 classes, softmax for probabilities)
    layers.Dense(10, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Display the model architecture summary
model.summary()

In [ ]:
# Train the model for 10 epochs
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

In [ ]:
# Plot training and validation accuracy and loss over epochs
plt.figure(figsize=(12, 4))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Make predictions on the first 5 test images
predictions = model.predict(test_images[:5])

# Define a function to display images and predictions
def plot_image(i, predictions_array, true_label, img, class_names):
    # Ensure correct indexing and shapes
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]

    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)

    # Get the predicted label index
    predicted_label = np.argmax(predictions_array)

    # Color-code the prediction: blue for correct, red for incorrect
    color = 'blue' if predicted_label == true_label[0] else 'red'

    # Set the label text
    plt.xlabel(f"Predicted: {class_names[predicted_label]} (True: {class_names[true_label[0]]})",
               color=color)

# Visualize the first 5 test images with predictions
plt.figure(figsize=(5, 10))
for i in range(5):
    plt.subplot(5, 1, i + 1)
    plot_image(i, predictions, test_labels, test_images, class_names)

plt.tight_layout()
plt.show()

In [ ]:
# --- Implementation of Tasks for Improvement ---
print("\n--- Starting Experiment: Modified CNN with slightly reduced complexity ---")

# 1. Define the Modified CNN Model with fewer layers and dropout
model_improved = models.Sequential([
    # Initial Feature Extraction with more filters
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'), # Reduced filters
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'), # Increased filters in the second layer
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'), # Increased filters
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Flatten for Dense Layers
    layers.Flatten(),

    # Dropout for Regularization (to fight overfitting)
    layers.Dropout(0.5),

    # Classification Head with more neurons
    layers.Dense(256, activation='relu'), # Increased neurons
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# 2. Compile the Modified Model
model_improved.compile(optimizer='adam',
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                       metrics=['accuracy'])

model_improved.summary()

# 3. Train the Modified Model with 10 epochs and Early Stopping
EPOCHS = 10
callbacks_improved = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, mode='max', baseline=0.90)
]

history_improved = model_improved.fit(train_images, train_labels, epochs=EPOCHS,
                                      validation_data=(test_images, test_labels),
                                      callbacks=callbacks_improved,
                                      verbose=1)

# 4. Evaluate the Modified Model
test_loss_improved, test_acc_improved = model_improved.evaluate(test_images, test_labels, verbose=2)
print(f"\nImproved Model Test Accuracy: {test_acc_improved:.4f}")

# 5. Plot the Improved Model's history
plt.figure(figsize=(12, 4))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history_improved.history['accuracy'], label='Training Accuracy')
plt.plot(history_improved.history['val_accuracy'], label='Validation Accuracy')
plt.title('Improved Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history_improved.history['loss'], label='Training Loss')
plt.plot(history_improved.history['val_loss'], label='Validation Loss')
plt.title('Improved Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Define a function to display images and predictions
def plot_image(i, predictions_array, true_label, img, class_names):
    # Ensure correct indexing and shapes
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]

    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)

    # Get the predicted label index
    predicted_label = np.argmax(predictions_array)

    # Color-code the prediction: blue for correct, red for incorrect
    color = 'blue' if predicted_label == true_label[0] else 'red'

    # Set the label text
    plt.xlabel(f"Predicted: {class_names[predicted_label]} (True: {class_names[true_label[0]]})",
               color=color)

In [ ]:
# Make predictions on the first 5 test images using the improved model
predictions_improved = model_improved.predict(test_images[:5])

# Visualize the first 5 test images with predictions from the improved model
plt.figure(figsize=(5, 10))
for i in range(5):
    plt.subplot(5, 1, i + 1)
    # Use the plot_image function defined earlier
    plot_image(i, predictions_improved, test_labels, test_images, class_names)

plt.tight_layout()
plt.show()